In [ ]:
import os
import scipy.io
import re
import numpy as np
from scipy.stats import kurtosis, skew
import matplotlib.pyplot as plt
import pywt


In [2]:
# Main directory
main_dir = r'D:\Grinding Machine\Vibration_Data\2nd_round'

In [4]:
# List of specific keys
keys = ['BF', 'BL', 'N', 'TF']

# Initialize the mat_data dictionary with these keys
mat_data = {key: [] for key in keys}

In [ ]:
mat_data

In [6]:
# Iterate through each subdirectory in the main directory
for sub_dir in os.listdir(main_dir):
    # Construct the path to the subdirectory
    sub_dir_path = os.path.join(main_dir, sub_dir)

    # Check if it's a directory
    if os.path.isdir(sub_dir_path):
        # Iterate through each file in the subdirectory
        for file in os.listdir(sub_dir_path):
            # Check if the file is a .mat file
            if file.endswith('.mat'):
                # Construct the full file path
                file_path = os.path.join(sub_dir_path, file)

                # Load the .mat file
                mat_contents = scipy.io.loadmat(file_path)
                

                signal = mat_contents['signals'][0]


             
                mat_data[sub_dir].append(signal)
                

# Now mat_data contains the contents of all .mat files in the subdirectories


In [ ]:
len(mat_data['BF'])

In [8]:
for key in mat_data.keys():
    # This concatenates all arrays in the list into one array, then flattens it
    mat_data[key] = np.concatenate(mat_data[key]).flatten()

In [ ]:
print(len(mat_data['BF']))

In [10]:
segment_length = 25600  # Length of each segment
overlap = int(segment_length * 0.35)  # 75% overlap

segmented_data = {}  # Dictionary to store segmented signals

for key, signal in mat_data.items():
    # Initialize a list to store segments for this key
    segments = []
    
    # Start index for the first segment
    start_idx = 0
    
    # Calculate the number of segments we can extract
    while start_idx + segment_length <= len(signal):
        # Extract a segment
        segment = signal[start_idx:start_idx + segment_length]
        segments.append(segment)
        
        # Update start index for the next segment, accounting for the new overlap
        # This effectively moves the start index by 25% of the segment length each time
        start_idx += segment_length - overlap  # Move start index by 25% of segment length for 75% overlap
    
    # Store the list of segments in the new dictionary
    segmented_data[key] = segments


In [ ]:
len(segmented_data['BF'])

In [12]:
def create_scalogram(signal, fs):
    scales = np.arange(1, 128)
    wavelet = 'morl'
    coefficients, _ = pywt.cwt(signal, scales, wavelet, 1.0 / fs)
    return np.abs(coefficients)

In [ ]:
main_dir = 'D:\Grinding Machine\Vib CWT'  # Replace with your main directory path # Initialize your dictionary


# Your existing code to populate mat_data...

# Now, iterate over the dictionary and create scalograms

scales = np.arange(1, 128)
for class_name, signals in segmented_data.items():
    class_dir = os.path.join(main_dir, class_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    for i, signal in enumerate(signals):

        scalogram = create_scalogram(signal, 25600)  # Replace 25600 with actual fs if different
        plt.imshow(scalogram, extent=[0, len(signal) / 25600, 1, max(scales)], cmap='jet', aspect='auto')
        plt.axis('off')
        plt.tight_layout()
        print(i)


        # Save the figure
        plt.savefig(os.path.join(class_dir, f'scalogram_2nd_round_{i}.png'))
        plt.close()